In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: NVIDIA GeForce RTX 3060 Ti


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
import emoji
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /home/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
#Clean emojis from text
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

def strip_emoji(text):
    return re.sub(get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, stopwords, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)
    text =' '.join(word for word in text.split() if len(word) < 14) # remove words longer than 14 characters
    return text

#remove contractions
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the "#" symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as "&" and "$" present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

#Remove multiple sequential spaces
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)

#Stemming
def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

#Lemmatization 
def lemmatize(text):
    tokenized = nltk.word_tokenize(text)
    lm = WordNetLemmatizer()
    return ' '.join([lm.lemmatize(words) for words in tokenized])

def deep_clean(text):
    # text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    # text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    text = stemmer(text)
    return text

In [33]:
def preprocess_csv(input_path, output_path):    
    df = pd.read_csv(input_path,  encoding='latin-1', names=['target', 'question', 'content', 'answer'])
    print('Number of dataset sentences: {:,}\n'.format(df.shape[0]))
    print(df['target'].value_counts())
    df = df.fillna('')
    df['text'] = df['question'].map(str) + ' ' + df['content'].map(str) + ' ' + df['answer'].map(str)
    df = df[['target', 'text']]
    #Get the cleaned tweets
    texts_new = []
    i = 0
    for t in df.text:
        i = i+1
        texts_new.append(deep_clean(t))
    df['text_clean'] = texts_new
    df.drop_duplicates("text_clean", inplace=True)
    df = df[['target', 'text_clean']]
    df.to_csv(output_path, encoding='utf-8')

In [34]:
preprocess_csv("./data/yahoo_answers_csv/test.csv", "./data/yahoo_answers_csv/test_cleaned.csv")
preprocess_csv("./data/yahoo_answers_csv/train.csv", "./data/yahoo_answers_csv/train_cleaned.csv")

Number of dataset sentences: 60,000

9     6000
2     6000
4     6000
3     6000
5     6000
8     6000
7     6000
10    6000
1     6000
6     6000
Name: target, dtype: int64
Number of dataset sentences: 1,400,000

5     140000
6     140000
3     140000
7     140000
2     140000
8     140000
4     140000
9     140000
10    140000
1     140000
Name: target, dtype: int64


In [3]:
def generate_dataset(df):
    sentences = df.text_clean.values
    labels = df.target.values
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.\
                            truncation=True,
                            add_special_tokens=True, # Add '[CLS]' and '[SEP]'
                            max_length=70,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask=True,   # Construct attn. masks.
                            return_tensors='pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
        # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    dataset = TensorDataset(input_ids, attention_masks, labels)
    return dataset

In [36]:
test_df = pd.read_csv("./data/yahoo_answers_csv/test_cleaned.csv")
test_df = test_df[test_df['text_clean'].notnull()] 
test_df['target'] = test_df['target'] - 1
test_dataset = generate_dataset(test_df)

train_df = pd.read_csv("./data/yahoo_answers_csv/train_cleaned.csv")
train_df = train_df[train_df['text_clean'].notnull()] 
train_df['target'] = train_df['target'] - 1
train_dataset = generate_dataset(train_df)

In [37]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [38]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 10, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    return_dict=False,
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [39]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [40]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/home/marco/anaconda3/envs/CMPT_713_final_project/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [41]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [42]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [43]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [44]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  43,726.    Elapsed: 0:00:07.
  Batch    80  of  43,726.    Elapsed: 0:00:14.
  Batch   120  of  43,726.    Elapsed: 0:00:22.
  Batch   160  of  43,726.    Elapsed: 0:00:29.
  Batch   200  of  43,726.    Elapsed: 0:00:36.
  Batch   240  of  43,726.    Elapsed: 0:00:43.
  Batch   280  of  43,726.    Elapsed: 0:00:51.
  Batch   320  of  43,726.    Elapsed: 0:00:58.
  Batch   360  of  43,726.    Elapsed: 0:01:05.
  Batch   400  of  43,726.    Elapsed: 0:01:13.
  Batch   440  of  43,726.    Elapsed: 0:01:20.
  Batch   480  of  43,726.    Elapsed: 0:01:27.
  Batch   520  of  43,726.    Elapsed: 0:01:35.
  Batch   560  of  43,726.    Elapsed: 0:01:42.
  Batch   600  of  43,726.    Elapsed: 0:01:49.
  Batch   640  of  43,726.    Elapsed: 0:01:57.
  Batch   680  of  43,726.    Elapsed: 0:02:04.
  Batch   720  of  43,726.    Elapsed: 0:02:11.
  Batch   760  of  43,726.    Elapsed: 0:02:18.
  Batch   800  of  43,726.    Elapsed: 0:02:2

  Batch 6,840  of  43,726.    Elapsed: 0:20:53.
  Batch 6,880  of  43,726.    Elapsed: 0:21:00.
  Batch 6,920  of  43,726.    Elapsed: 0:21:08.
  Batch 6,960  of  43,726.    Elapsed: 0:21:15.
  Batch 7,000  of  43,726.    Elapsed: 0:21:22.
  Batch 7,040  of  43,726.    Elapsed: 0:21:30.
  Batch 7,080  of  43,726.    Elapsed: 0:21:37.
  Batch 7,120  of  43,726.    Elapsed: 0:21:44.
  Batch 7,160  of  43,726.    Elapsed: 0:21:52.
  Batch 7,200  of  43,726.    Elapsed: 0:21:59.
  Batch 7,240  of  43,726.    Elapsed: 0:22:06.
  Batch 7,280  of  43,726.    Elapsed: 0:22:14.
  Batch 7,320  of  43,726.    Elapsed: 0:22:21.
  Batch 7,360  of  43,726.    Elapsed: 0:22:28.
  Batch 7,400  of  43,726.    Elapsed: 0:22:36.
  Batch 7,440  of  43,726.    Elapsed: 0:22:43.
  Batch 7,480  of  43,726.    Elapsed: 0:22:50.
  Batch 7,520  of  43,726.    Elapsed: 0:22:58.
  Batch 7,560  of  43,726.    Elapsed: 0:23:05.
  Batch 7,600  of  43,726.    Elapsed: 0:23:12.
  Batch 7,640  of  43,726.    Elapsed: 0

  Batch 13,600  of  43,726.    Elapsed: 0:41:36.
  Batch 13,640  of  43,726.    Elapsed: 0:41:44.
  Batch 13,680  of  43,726.    Elapsed: 0:41:51.
  Batch 13,720  of  43,726.    Elapsed: 0:41:58.
  Batch 13,760  of  43,726.    Elapsed: 0:42:06.
  Batch 13,800  of  43,726.    Elapsed: 0:42:13.
  Batch 13,840  of  43,726.    Elapsed: 0:42:21.
  Batch 13,880  of  43,726.    Elapsed: 0:42:28.
  Batch 13,920  of  43,726.    Elapsed: 0:42:35.
  Batch 13,960  of  43,726.    Elapsed: 0:42:43.
  Batch 14,000  of  43,726.    Elapsed: 0:42:50.
  Batch 14,040  of  43,726.    Elapsed: 0:42:57.
  Batch 14,080  of  43,726.    Elapsed: 0:43:05.
  Batch 14,120  of  43,726.    Elapsed: 0:43:12.
  Batch 14,160  of  43,726.    Elapsed: 0:43:19.
  Batch 14,200  of  43,726.    Elapsed: 0:43:27.
  Batch 14,240  of  43,726.    Elapsed: 0:43:34.
  Batch 14,280  of  43,726.    Elapsed: 0:43:42.
  Batch 14,320  of  43,726.    Elapsed: 0:43:49.
  Batch 14,360  of  43,726.    Elapsed: 0:43:56.
  Batch 14,400  of  

  Batch 20,320  of  43,726.    Elapsed: 1:02:13.
  Batch 20,360  of  43,726.    Elapsed: 1:02:21.
  Batch 20,400  of  43,726.    Elapsed: 1:02:28.
  Batch 20,440  of  43,726.    Elapsed: 1:02:35.
  Batch 20,480  of  43,726.    Elapsed: 1:02:43.
  Batch 20,520  of  43,726.    Elapsed: 1:02:50.
  Batch 20,560  of  43,726.    Elapsed: 1:02:57.
  Batch 20,600  of  43,726.    Elapsed: 1:03:05.
  Batch 20,640  of  43,726.    Elapsed: 1:03:12.
  Batch 20,680  of  43,726.    Elapsed: 1:03:19.
  Batch 20,720  of  43,726.    Elapsed: 1:03:27.
  Batch 20,760  of  43,726.    Elapsed: 1:03:34.
  Batch 20,800  of  43,726.    Elapsed: 1:03:42.
  Batch 20,840  of  43,726.    Elapsed: 1:03:49.
  Batch 20,880  of  43,726.    Elapsed: 1:03:56.
  Batch 20,920  of  43,726.    Elapsed: 1:04:04.
  Batch 20,960  of  43,726.    Elapsed: 1:04:11.
  Batch 21,000  of  43,726.    Elapsed: 1:04:18.
  Batch 21,040  of  43,726.    Elapsed: 1:04:26.
  Batch 21,080  of  43,726.    Elapsed: 1:04:33.
  Batch 21,120  of  

  Batch 27,040  of  43,726.    Elapsed: 1:22:50.
  Batch 27,080  of  43,726.    Elapsed: 1:22:57.
  Batch 27,120  of  43,726.    Elapsed: 1:23:05.
  Batch 27,160  of  43,726.    Elapsed: 1:23:12.
  Batch 27,200  of  43,726.    Elapsed: 1:23:19.
  Batch 27,240  of  43,726.    Elapsed: 1:23:27.
  Batch 27,280  of  43,726.    Elapsed: 1:23:34.
  Batch 27,320  of  43,726.    Elapsed: 1:23:41.
  Batch 27,360  of  43,726.    Elapsed: 1:23:49.
  Batch 27,400  of  43,726.    Elapsed: 1:23:56.
  Batch 27,440  of  43,726.    Elapsed: 1:24:03.
  Batch 27,480  of  43,726.    Elapsed: 1:24:11.
  Batch 27,520  of  43,726.    Elapsed: 1:24:18.
  Batch 27,560  of  43,726.    Elapsed: 1:24:26.
  Batch 27,600  of  43,726.    Elapsed: 1:24:33.
  Batch 27,640  of  43,726.    Elapsed: 1:24:40.
  Batch 27,680  of  43,726.    Elapsed: 1:24:48.
  Batch 27,720  of  43,726.    Elapsed: 1:24:55.
  Batch 27,760  of  43,726.    Elapsed: 1:25:02.
  Batch 27,800  of  43,726.    Elapsed: 1:25:10.
  Batch 27,840  of  

...

  Batch 13,560  of  43,726.    Elapsed: 0:41:46.
  Batch 13,600  of  43,726.    Elapsed: 0:41:53.
  Batch 13,640  of  43,726.    Elapsed: 0:42:01.
  Batch 13,680  of  43,726.    Elapsed: 0:42:08.
  Batch 13,720  of  43,726.    Elapsed: 0:42:15.
  Batch 13,760  of  43,726.    Elapsed: 0:42:23.
  Batch 13,800  of  43,726.    Elapsed: 0:42:30.
  Batch 13,840  of  43,726.    Elapsed: 0:42:37.
  Batch 13,880  of  43,726.    Elapsed: 0:42:45.
  Batch 13,920  of  43,726.    Elapsed: 0:42:52.
  Batch 13,960  of  43,726.    Elapsed: 0:43:00.
  Batch 14,000  of  43,726.    Elapsed: 0:43:07.
  Batch 14,040  of  43,726.    Elapsed: 0:43:14.
  Batch 14,080  of  43,726.    Elapsed: 0:43:22.
  Batch 14,120  of  43,726.    Elapsed: 0:43:29.
  Batch 14,160  of  43,726.    Elapsed: 0:43:36.
  Batch 14,200  of  43,726.    Elapsed: 0:43:44.
  Batch 14,240  of  43,726.    Elapsed: 0:43:51.
  Batch 14,280  of  43,726.    Elapsed: 0:43:58.
  Batch 14,320  of  43,726.    Elapsed: 0:44:06.
  Batch 14,360  of  

  Batch 20,280  of  43,726.    Elapsed: 1:02:27.
  Batch 20,320  of  43,726.    Elapsed: 1:02:35.
  Batch 20,360  of  43,726.    Elapsed: 1:02:42.
  Batch 20,400  of  43,726.    Elapsed: 1:02:50.
  Batch 20,440  of  43,726.    Elapsed: 1:02:57.
  Batch 20,480  of  43,726.    Elapsed: 1:03:04.
  Batch 20,520  of  43,726.    Elapsed: 1:03:12.
  Batch 20,560  of  43,726.    Elapsed: 1:03:19.
  Batch 20,600  of  43,726.    Elapsed: 1:03:26.
  Batch 20,640  of  43,726.    Elapsed: 1:03:34.
  Batch 20,680  of  43,726.    Elapsed: 1:03:41.
  Batch 20,720  of  43,726.    Elapsed: 1:03:49.
  Batch 20,760  of  43,726.    Elapsed: 1:03:56.
  Batch 20,800  of  43,726.    Elapsed: 1:04:03.
  Batch 20,840  of  43,726.    Elapsed: 1:04:11.
  Batch 20,880  of  43,726.    Elapsed: 1:04:18.
  Batch 20,920  of  43,726.    Elapsed: 1:04:26.
  Batch 20,960  of  43,726.    Elapsed: 1:04:33.
  Batch 21,000  of  43,726.    Elapsed: 1:04:41.
  Batch 21,040  of  43,726.    Elapsed: 1:04:48.
  Batch 21,080  of  

  Batch 27,000  of  43,726.    Elapsed: 1:23:14.
  Batch 27,040  of  43,726.    Elapsed: 1:23:22.
  Batch 27,080  of  43,726.    Elapsed: 1:23:29.
  Batch 27,120  of  43,726.    Elapsed: 1:23:36.
  Batch 27,160  of  43,726.    Elapsed: 1:23:44.
  Batch 27,200  of  43,726.    Elapsed: 1:23:51.
  Batch 27,240  of  43,726.    Elapsed: 1:23:59.
  Batch 27,280  of  43,726.    Elapsed: 1:24:06.
  Batch 27,320  of  43,726.    Elapsed: 1:24:13.
  Batch 27,360  of  43,726.    Elapsed: 1:24:21.
  Batch 27,400  of  43,726.    Elapsed: 1:24:28.
  Batch 27,440  of  43,726.    Elapsed: 1:24:36.
  Batch 27,480  of  43,726.    Elapsed: 1:24:43.
  Batch 27,520  of  43,726.    Elapsed: 1:24:50.
  Batch 27,560  of  43,726.    Elapsed: 1:24:58.
  Batch 27,600  of  43,726.    Elapsed: 1:25:05.
  Batch 27,640  of  43,726.    Elapsed: 1:25:13.
  Batch 27,680  of  43,726.    Elapsed: 1:25:20.
  Batch 27,720  of  43,726.    Elapsed: 1:25:28.
  Batch 27,760  of  43,726.    Elapsed: 1:25:35.
  Batch 27,800  of  

  Batch 33,720  of  43,726.    Elapsed: 1:44:04.
  Batch 33,760  of  43,726.    Elapsed: 1:44:12.
  Batch 33,800  of  43,726.    Elapsed: 1:44:20.
  Batch 33,840  of  43,726.    Elapsed: 1:44:27.
  Batch 33,880  of  43,726.    Elapsed: 1:44:34.
  Batch 33,920  of  43,726.    Elapsed: 1:44:42.
  Batch 33,960  of  43,726.    Elapsed: 1:44:49.
  Batch 34,000  of  43,726.    Elapsed: 1:44:57.
  Batch 34,040  of  43,726.    Elapsed: 1:45:05.
  Batch 34,080  of  43,726.    Elapsed: 1:45:12.
  Batch 34,120  of  43,726.    Elapsed: 1:45:20.
  Batch 34,160  of  43,726.    Elapsed: 1:45:27.
  Batch 34,200  of  43,726.    Elapsed: 1:45:35.
  Batch 34,240  of  43,726.    Elapsed: 1:45:42.
  Batch 34,280  of  43,726.    Elapsed: 1:45:49.
  Batch 34,320  of  43,726.    Elapsed: 1:45:57.
  Batch 34,360  of  43,726.    Elapsed: 1:46:04.
  Batch 34,400  of  43,726.    Elapsed: 1:46:12.
  Batch 34,440  of  43,726.    Elapsed: 1:46:19.
  Batch 34,480  of  43,726.    Elapsed: 1:46:27.
  Batch 34,520  of  

  Batch 40,440  of  43,726.    Elapsed: 2:04:57.
  Batch 40,480  of  43,726.    Elapsed: 2:05:04.
  Batch 40,520  of  43,726.    Elapsed: 2:05:12.
  Batch 40,560  of  43,726.    Elapsed: 2:05:19.
  Batch 40,600  of  43,726.    Elapsed: 2:05:26.
  Batch 40,640  of  43,726.    Elapsed: 2:05:34.
  Batch 40,680  of  43,726.    Elapsed: 2:05:41.
  Batch 40,720  of  43,726.    Elapsed: 2:05:48.
  Batch 40,760  of  43,726.    Elapsed: 2:05:56.
  Batch 40,800  of  43,726.    Elapsed: 2:06:03.
  Batch 40,840  of  43,726.    Elapsed: 2:06:11.
  Batch 40,880  of  43,726.    Elapsed: 2:06:18.
  Batch 40,920  of  43,726.    Elapsed: 2:06:26.
  Batch 40,960  of  43,726.    Elapsed: 2:06:33.
  Batch 41,000  of  43,726.    Elapsed: 2:06:40.
  Batch 41,040  of  43,726.    Elapsed: 2:06:48.
  Batch 41,080  of  43,726.    Elapsed: 2:06:55.
  Batch 41,120  of  43,726.    Elapsed: 2:07:03.
  Batch 41,160  of  43,726.    Elapsed: 2:07:10.
  Batch 41,200  of  43,726.    Elapsed: 2:07:17.
  Batch 41,240  of  

In [45]:
torch.save(model.state_dict(), "./model/BERT.pt")